In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import random

random.seed()

CARDLEVELVAL = 10000

In [ ]:
# 随机生成卡牌
def gencard_normal(cardtypearr, cardlevelarr, scenearr, gentimes, startcost):
    cri = int(random.random() * len(cardtypearr))
    cli = int(random.random() * len(cardlevelarr))
    
    return makeCard(cardtypearr[cri], cardlevelarr[cli]), startcost

def getGenCardIndex(scenearr):
    for i in range(len(scenearr)):
        if scenearr[i] == 0:
            return i
    
    return -1

def hasCard(scenearr, card):
    for i in range(len(scenearr)):
        if scenearr[i] == card:
            return True
        
    return False

def hasCardLevel(scenearr, cardlevel):
    for i in range(len(scenearr)):
        if scenearr[i] % CARDLEVELVAL == cardlevel:
            return True
        
    return False

def hasAllCardLevel(scenearr, cardtypearr, cardlevel):
    for i in range(len(cardtypearr)):
        if not hasCard(scenearr, makeCard(cardtypearr[i], cardlevel)):
            return False
        
    return True

def countCardNums(scenearr, card):
    nums = 0
    
    for i in range(len(scenearr)):
        if scenearr[i] == card:
            nums = nums + 1
        
    return nums

def makeCard(cardtype, cardlevel):
    return cardtype * CARDLEVELVAL + cardlevel

def getMaxCardLevel(scenearr):
    level = 0
    
    for i in range(len(scenearr)):
        cl = scenearr[i] % CARDLEVELVAL
        if cl > level:
            level = cl
            
    
    return level

def getMinLevelCard(scenearr):
    level = 0
    c = 0
    
    for i in range(len(scenearr)):
        if scenearr[i] > 0:
            cl = scenearr[i] % CARDLEVELVAL
            if level <= 0 or level > cl:
                level = cl
                c = scenearr[i]            
    
    return c

def countCardNumsWithLevel(scenearr, level):
    nums = 0
    
    for i in range(len(scenearr)):
        if scenearr[i] % CARDLEVELVAL == level:
            nums = nums + 1
        
    return nums

def comboCards(scenearr, maxcardlevel):
    for i in range(len(scenearr) - 1):
        if scenearr[i] == 0:
            continue
        
        for j in range(i + 1, len(scenearr)):
            if i != j and scenearr[i] == scenearr[j]:
                ct = int(scenearr[i] / CARDLEVELVAL)
                cl = scenearr[i] % CARDLEVELVAL
                
                if cl < maxcardlevel:
                    scenearr[i] = ct * CARDLEVELVAL + cl + 1
                    scenearr[j] = 0

                    return True
    
    return False

def countLevelOneCardNums(card):
    cl = card % CARDLEVELVAL
    return pow(2, (cl - 1))

def countLevelOneCardNumsList(scenearr):
    n = 0
    for i in range(len(scenearr)):
        if scenearr[i] > 0:
            n += countLevelOneCardNums(scenearr[i])
            
    return n

def getFreeBlockNums(scenearr):
    n = 0
    for v in scenearr:
        if v == 0:
            n += 1
    
    return n

def procScene(funcGen, cardtypearr, cardlevelarr, maxcardlevel, scenearr, gentimes, startcost):
    ci = getGenCardIndex(scenearr)
    if ci < 0:
        return False, 0, 0
    
    card, cost = funcGen(cardtypearr, cardlevelarr, scenearr, gentimes, startcost)
    scenearr[ci] = card
    
    combonums = 0
    
    while True is True:
        r = comboCards(scenearr, maxcardlevel)
#         print(r)
        if not r:
            break
            
        combonums = combonums + 1
    
    return True, cost, combonums

In [ ]:
def showBars3(names, xarrs, yarrs):
    fig = go.Figure()
    
    for i in range(len(names)):
        fig.add_trace(go.Bar(
            x=xarrs[i],
            y=yarrs[i],
            name=names[i],
        ))            

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.show()

In [ ]:
tags = []
winnums = []
nums10 = []
gennums = []
gennums2 = []
maxcardlevel = []
maxlevelnums = []
combonums = []

for n in range(2, 16):
    gentimes = 0
    gentimes2 = 0
    scenearr = [0 for i in range(n)]
    cost = 0
    mcl = 0
    mln = 0
    curcn = 0

    while True is True:
        r, cc, cnn = procScene(gencard_normal, [1], [1], 10, scenearr, gentimes, 1)
        if not r:
            mcl = getMaxCardLevel(scenearr)
            mln = countCardNumsWithLevel(scenearr, mcl)
            
            break

#         if hasAllCardLevel(scenearr, [1], 10):
#             gentimes = gentimes + 1
#             cost = cost + cc
#         else:
#             gentimes2 = gentimes2 + 1
        gentimes = gentimes + 1
        cost = cost + cc      
        curcn = curcn + cnn
    
    cn = countCardNums(scenearr, makeCard(1, 10))
    tags.append('{}个格子'.format(n))
    if cn > 0:
        winnums.append(1)
    else:
        winnums.append(0)
        
    nums10.append(cn)
    
    gennums.append(gentimes)
#     gennums2.append(gentimes2 + 1)    
    
    maxcardlevel.append(mcl)
    maxlevelnums.append(mln)
    combonums.append(curcn)

    
showBars3(['是否可以合成到10级', '最高级', '最高级卡数量', '全占满需要的卡牌数', '合成次数'],
         [tags, tags, tags, tags, tags],
         [winnums, maxcardlevel, maxlevelnums, gennums, combonums])

In [ ]:
tags = []
winnums = []
nums10 = []
gennums = []
gennums2 = []
maxcardlevel = []
maxlevelnums = []
combonums = []
maxper = []

for n in range(2, 16):
    totalgentimes = 0
#     gentimes2 = 0
    scenearr = [0 for i in range(n)]
    totalcost = 0
    mcl = 0
    mln = 0
    mclnums = 0
    curcn = 0

    curtimes = 30
    for tti in range(curtimes):
#         print(gentimes)
        scenearr = [0 for i in range(n)]
        gentimes = 0
        cost = 0
    
        while True is True:
            r, cc, cnn = procScene(gencard_normal, [1], [1, 2, 3], 10, scenearr, gentimes, 1)
            if not r:
                curmcl = getMaxCardLevel(scenearr)
                curmln = countCardNumsWithLevel(scenearr, curmcl)
                
                if curmcl > mcl:
                    mcl = curmcl
                    mln = curmln
                    mclnums = 1
                else:
                    mln += curmln
                    mclnums = mclnums + 1

                break

            gentimes = gentimes + 1
            cost = cost + cc      
            curcn = curcn + cnn
            
        totalgentimes = totalgentimes + gentimes
        totalcost = totalcost + cost        
    
#     cn = countCardNums(scenearr, makeCard(1, 10))
    tags.append('{}个格子'.format(n))
#     if cn > 0:
#         winnums.append(1)
#     else:
#         winnums.append(0)
        
#     nums10.append(cn)
    
    gennums.append(totalgentimes / curtimes)
#     gennums2.append(gentimes2 + 1)    
    
    maxcardlevel.append(mcl)
    maxlevelnums.append(mln / mclnums)
    combonums.append(curcn / curtimes)
    maxper.append(mclnums / curtimes)

    
showBars3(['最高级', '最高级卡数量', '全占满需要的卡牌数', '合成次数', '合成到最高级的概率'],
         [tags, tags, tags, tags, tags],
         [maxcardlevel, maxlevelnums, gennums, combonums, maxper])

In [ ]:
tags = []
winnums = []
nums10 = []
gennums = []
gennums2 = []
maxcardlevel = []
maxlevelnums = []
combonums = []


n = 15
gentimes = 0
gentimes2 = 0
scenearr = [0 for i in range(n)]
cost = 0
mcl = 0
mln = 0
curcn = 0

while True is True:
    r, cc, cnn = procScene(gencard_normal, [1], [1], 25, scenearr, gentimes, 1)
    if not r:
        mcl = getMaxCardLevel(scenearr)
        mln = countCardNumsWithLevel(scenearr, mcl)

        break

#         if hasAllCardLevel(scenearr, [1], 10):
#             gentimes = gentimes + 1
#             cost = cost + cc
#         else:
#             gentimes2 = gentimes2 + 1
    gentimes = gentimes + 1
    cost = cost + cc      
    curcn = curcn + cnn

cn = countCardNums(scenearr, makeCard(1, 10))
tags.append('{}个格子'.format(n))
if cn > 0:
    winnums.append(1)
else:
    winnums.append(0)

nums10.append(cn)

gennums.append(gentimes)
#     gennums2.append(gentimes2 + 1)    

maxcardlevel.append(mcl)
maxlevelnums.append(mln)
combonums.append(curcn)

    
showBars3(['是否可以合成到10级', '最高级', '最高级卡数量', '全占满需要的卡牌数', '合成次数'],
         [tags, tags, tags, tags, tags],
         [winnums, maxcardlevel, maxlevelnums, gennums, combonums])

In [ ]:
# 随机生成卡牌，始终生成距离最大等级一定等级的卡牌
def gencard_level(cardtypearr, cardlevelarr, scenearr, gentimes, startcost):
    mcl = getMaxCardLevel(scenearr)
    clr = int(len(scenearr) / len(cardtypearr))
    if mcl >= clr:
        if getFreeBlockNums(scenearr) == 1:
            mincard = getMinLevelCard(scenearr)
            if mincard > 0:
                return mincard, startcost * 2 * mcl * (mcl + 1)
            
        cri = int(random.random() * len(cardtypearr))
        cli = int(random.random() * len(cardlevelarr))
        
        if hasCard(scenearr, makeCard(cardtypearr[cri], cardlevelarr[cli])):
            return makeCard(cardtypearr[cri], cardlevelarr[cli]), startcost * 2 * mcl * (mcl + 1)

        return makeCard(cardtypearr[cri], cardlevelarr[cli] + mcl - clr + 1), startcost * 2 * mcl * (mcl + 1)

    cri = int(random.random() * len(cardtypearr))
    cli = int(random.random() * len(cardlevelarr))
    
    return makeCard(cardtypearr[cri], cardlevelarr[cli]), startcost * 2 * mcl * (mcl + 1)

In [ ]:
card, cost = gencard_level([1], [1, 2], [makeCard(1, 2), 0], 1, 1)

card, cost

In [ ]:
tags = []
winnums = []
nums10 = []
gennums = []
gennums2 = []
maxcardlevel = []
maxlevelnums = []
combonums = []
l1nums = []


for n in range(2, 16):
    gentimes = 0
    gentimes2 = 0
    scenearr = [0 for i in range(n)]
    cost = 0
    mcl = 0
    mln = 0
    curcn = 0
    curl1nums = 0

    while True is True:
        r, cc, cnn = procScene(gencard_level, [1], [1], 100, scenearr, gentimes, 1)
        if not r:
            mcl = getMaxCardLevel(scenearr)
            mln = countCardNumsWithLevel(scenearr, mcl)

            break

    #         if hasAllCardLevel(scenearr, [1], 10):
    #             gentimes = gentimes + 1
    #             cost = cost + cc
    #         else:
    #             gentimes2 = gentimes2 + 1
        gentimes = gentimes + 1
        cost = cost + cc      
        curcn = curcn + cnn
        
    curl1nums = countLevelOneCardNumsList(scenearr)
    l1nums.append(curl1nums)

    cn = countCardNums(scenearr, makeCard(1, 10))
    tags.append('{}个格子'.format(n))
    if cn > 0:
        winnums.append(1)
    else:
        winnums.append(0)

    nums10.append(cn)

    gennums.append(gentimes)
    #     gennums2.append(gentimes2 + 1)    

    maxcardlevel.append(mcl)
    maxlevelnums.append(mln)
    combonums.append(curcn)

    
showBars3(['最高级', '最高级卡数量', '全占满需要的卡牌数', '合成次数', '需要的1级卡数量'],
         [tags, tags, tags, tags, tags],
         [maxcardlevel, maxlevelnums, gennums, combonums, l1nums])

In [ ]:
tags = []
winnums = []
nums10 = []
gennums = []
gennums2 = []
maxcardlevel = []
maxlevelnums = []
combonums = []
l1nums = []


for n in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
    gentimes = 0
    gentimes2 = 0
    scenearr = [0 for i in range(15)]
    cost = 0
    mcl = 0
    mln = 0
    curcn = 0
    curl1nums = 0

    while True is True:
        r, cc, cnn = procScene(gencard_level, [1], [1], n, scenearr, gentimes, 1)
        if not r:
            mcl = getMaxCardLevel(scenearr)
            mln = countCardNumsWithLevel(scenearr, mcl)

            break

    #             gentimes = gentimes + 1
    #             cost = cost + cc
    #         else:
    #             gentimes2 = gentimes2 + 1
        gentimes = gentimes + 1
        cost = cost + cc      
        curcn = curcn + cnn
        
        if hasAllCardLevel(scenearr, [1], n):
            mcl = getMaxCardLevel(scenearr)
            mln = countCardNumsWithLevel(scenearr, mcl)
            
            break        
        
    curl1nums = countLevelOneCardNumsList(scenearr)
    l1nums.append(curl1nums)

#     cn = countCardNums(scenearr, makeCard(1, 10))
    tags.append('最高{}级'.format(n))
#     if cn > 0:
#         winnums.append(1)
#     else:
#         winnums.append(0)

#     nums10.append(cn)

    gennums.append(gentimes)
    #     gennums2.append(gentimes2 + 1)    

    maxcardlevel.append(mcl)
    maxlevelnums.append(mln)
    combonums.append(curcn)

    
showBars3(['最高级', '最高级卡数量', '全占满需要的卡牌数', '合成次数', '需要的1级卡数量'],
         [tags, tags, tags, tags, tags],
         [maxcardlevel, maxlevelnums, gennums, combonums, l1nums])

In [ ]:
def procAllScene(funcGenCard, arrCardType, arrCardLevel, blocknums, startcost, maxcardlevel):
    gentimes = 0
    scenearr = [0 for i in range(blocknums)]
    cost = 0
    curcardnums = 0
    
    while True is True:
        r, cc, cnn = procScene(funcGenCard, arrCardType, arrCardLevel, maxcardlevel, scenearr, gentimes, startcost)
        if not r:
            break

        gentimes = gentimes + 1
        cost = cost + cc      
        curcardnums = curcardnums + cnn

        if hasAllCardLevel(scenearr, arrCardType, maxcardlevel):
            break
            
    return scenearr, gentimes, cost, curcardnums

In [ ]:
scenearr, gentimes, cost, cardnums = procAllScene(gencard_level, [1], [1, 2], 15, 1, 50)

scenearr, gentimes, cost, cardnums

In [ ]:
tags = []
winnums = []
nums10 = []
gennums = []
gennums2 = []
maxcardlevel = []
maxlevelnums = []
combonums = []
l1nums = []
lstcost = []


n = 50
for ln in range(1, 10):
    gentimes = 0
#     gentimes2 = 0
#     scenearr = [0 for i in range(15)]
    cost = 0
#     mcl = 0
#     mln = 0
    curcn = 0
#     curl1nums = 0
    startarr = [i + 1 for i in range(ln)]

    maxtimes = 10
    for ttn in range(maxtimes):
        scenearr, gt, cc, cn = procAllScene(gencard_level, [1], startarr, 15, 1, n)
        gentimes += gt
        cost += cc
        curcn += cn
#         while True is True:
#             r, cc, cnn = procScene(gencard_level, [1], startarr, n, scenearr, gentimes, 1)
#             if not r:
#     #             mcl = getMaxCardLevel(scenearr)
#     #             mln = countCardNumsWithLevel(scenearr, mcl)

#                 break

#         #             gentimes = gentimes + 1
#         #             cost = cost + cc
#         #         else:
#         #             gentimes2 = gentimes2 + 1
#             gentimes = gentimes + 1
#             cost = cost + cc      
#             curcn = curcn + cnn

#             if hasAllCardLevel(scenearr, [1], n):
#     #             mcl = getMaxCardLevel(scenearr)
#     #             mln = countCardNumsWithLevel(scenearr, mcl)

#                 break        
        
        curl1nums += countLevelOneCardNumsList(scenearr)

    l1nums.append(curl1nums / maxtimes)

#     cn = countCardNums(scenearr, makeCard(1, 10))
    tags.append('随机{}张卡'.format(ln))
#     if cn > 0:
#         winnums.append(1)
#     else:
#         winnums.append(0)

#     nums10.append(cn)

    gennums.append(gentimes / maxtimes)
    #     gennums2.append(gentimes2 + 1)    

#     maxcardlevel.append(mcl)
#     maxlevelnums.append(mln)
    combonums.append(curcn / maxtimes)
    lstcost.append(cost / maxtimes)

    
showBars3(['需要的卡牌数', '合成次数', '需要的1级卡数量', '抽卡成本'],
         [tags, tags, tags, tags],
         [gennums, combonums, l1nums, lstcost])

In [ ]:
def showLines3(names, xarrs, yarrs):
    fig = go.Figure()
    
    for i in range(len(names)):
        fig.add_trace(go.Scatter(x=xarrs[i], y=yarrs[i],
                    mode='lines',
                    name=names[i]))            

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.show()

In [ ]:
lstlevel = [n for n in range(1, 50)]
lstCost0 = [n for n in range(1, 50)]
lstCost0_1 = [2 * n for n in range(1, 50)]
lstCost1 = [n * n for n in range(1, 50)]
lstCost1_1 = [2 * n * (n + 1) for n in range(1, 50)]
lstCost2 = [n * n * n for n in range(1, 50)]
lstCost3 = [pow(2, n) for n in range(1, 50)]

showLines3(['n', '2n', 'n*n', '2n*(n+1)', 'n*n*n', 'pow(2, n)'],
         [lstlevel, lstlevel, lstlevel, lstlevel, lstlevel, lstlevel],
         [lstCost0, lstCost0_1, lstCost1, lstCost1_1, lstCost2, lstCost3])